In [1]:
import copy
import random as rd
import csv
import numpy as np
import pandas as pd
import math

In [2]:

# scenario = '02'
# metro = 'post'
walking_speed = 1  # m/s

routeTable = {}
roadLengthTable = {}
arrivalTime = {}
departuresProfile = {}
arrivalProfile = {}

def resetParams():
    routeTable = {}
    roadLengthTable = {}
    arrivalTime = {}
    departuresProfile = {}
    arrivalProfile = {}

def readRouteTable(metro):
    #Route table initialization
    with open(f'UptownInputs/roadCalcs_routeTable_{metro}.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            profileId = row[0]
            popSplit = float(row[2])
            route = list(filter(lambda x:x!='',row[3:]))
            if profileId not in routeTable:
                routeTable[profileId] = []
            routeTable[profileId].append(
                {
                    'popSplit': popSplit,
                    'route': route
                }
            )

def readRoadLengthTable():
#Route length initialization
    with open('UptownInputs/roadCalcs_roadLengthTable.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            roadProfile = row[0]
            length = float(row[1])
            roadLengthTable[roadProfile] = {
                'length': length,
                'walking_time': round(length / walking_speed) # Calculate walking time in seconds
            }

def readArrivalTime(scenario):
#population and arrival initialization
    def populationFormatter(pop):
        try:
            return 0 if pop == '' else int(pop.replace(',', ''))
        except:
            return 0
    with open(f'UptownInputs/roadCalcs_arrivalTime_{scenario}.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            profileId = row[0]
            time = float(row[3])
            population = populationFormatter(row[2])
            arrivalTime[profileId] = {
                'population': population,
                'time': time
            }

def readArrivalDepartureProfile(scenario):
#arrival profile
    with open(f'UptownInputs/roadCalcs_arrivalProfile_departures_{scenario}.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        data = list(reader)
        headers = data[0]
        arrivalData = data[1:]

        def pctFormatter(pct):
            try:
                return abs(float(pct))
            except:
                return abs(float(str(pct.split('%')[0])))/100

        for row in arrivalData:
            profileId = row[0]
            timeProfile = row[2:]
            departuresProfile[profileId] = {}
            for idx,pct in enumerate(timeProfile):
                departuresProfile[profileId][(idx*3600)] = 0 if pct == '' else pctFormatter(pct)


def readArrivalArrivalProfile(scenario):
#arrival profile
    with open(f'UptownInputs/roadCalcs_arrivalProfile_arrivals_{scenario}.csv', 'r') as csvfile:
        reader = csv.reader(csvfile)
        data = list(reader)
        headers = data[0]
        arrivalData = data[1:]

        def pctFormatter(pct):
            try:
                return abs(float(pct))
            except:
                return abs(float(str(pct.split('%')[0])))/100

        for row in arrivalData:
            profileId = row[0]
            timeProfile = row[2:]
            arrivalProfile[profileId] = {}
            for idx,pct in enumerate(timeProfile):
                arrivalProfile[profileId][(idx*3600)] = 0 if pct == '' else pctFormatter(pct)


In [3]:
def round_down(num, divisor):
    return num - (num%divisor)

In [4]:
class person:
    def __init__(self,personIndex,timeAtNode,pathList,nodeDetail,inNewSpace):
        self.idx = personIndex
        self.curNode = pathList[0]
        self.curNodeIdx = 0
        self.timeAtCurNode = timeAtNode
        self.nodeList = pathList
        self.node_detail = nodeDetail 
        self.timeAtNextNode = timeAtNode + self.calculateTimeToNextNode()
        self.agentInNewSpace = inNewSpace

    def __str__(self):
        return f"{self.idx} {self.curNode} {self.timeAtCurNode} {self.timeAtNextNode}"

    def calculateTimeToNextNode(self):
        # print(self.node_detail)
        timeToSpendAtCurNode = self.node_detail[self.curNode]['walking_time']
        return timeToSpendAtCurNode

    def updateAgentLocation(self,curTime):
        self.agentInNewSpace = False
        if curTime == self.timeAtNextNode:
            #Reached Last Step, remove details
            if self.curNode == self.nodeList[-1]:
                self.curNode = 'end'
                self.timeAtNextNode = -1
                return
    
            self.curNodeIdx+=1
            self.curNode = self.nodeList[self.curNodeIdx]
            self.timeAtCurNode = curTime
            self.timeAtNextNode = curTime + self.calculateTimeToNextNode()
            self.agentInNewSpace = True
            return

In [5]:
print(routeTable) #node paths
print(roadLengthTable)
print(arrivalTime)
print(departuresProfile)
print(arrivalProfile)

{}
{}
{}
{}
{}


In [ ]:



def runScenario(metro, scenario):
    startingTime = 0 * 3600  # 7pm in seconds
    simulationTime = 86400 + startingTime  # seconds
    peoplePerHour = 1500
    flowRateBucket = 60
    flowRateWidthTest = 66  # people per minute per metre

    nSeconds = simulationTime - startingTime
    nodes = list(roadLengthTable.keys()) + ["end"]
    nodeIndex = {node: i for i, node in enumerate(nodes)}

    # Preallocate arrays
    occupancy = np.zeros((nSeconds, len(nodes)), dtype=np.int32)
    peopleCount = np.zeros_like(occupancy)
    flowRate = np.zeros_like(occupancy)
    widthFlow = np.zeros_like(occupancy, dtype=float)

    # Track active people
    peopleList = []
    peopleEnd = 0

    # Precompute arrival/departure rates per (profile, routeId, bucket)
    precomputedRates = {}
    for profile in departuresProfile:
        if profile not in routeTable:
            continue
        precomputedRates[profile] = {}
        for routeId, routes in enumerate(routeTable[profile]):
            precomputedRates[profile][routeId] = {}
            for bucket, depVal in departuresProfile[profile].items():
                arrVal = arrivalProfile[profile][bucket]
                pop = arrivalTime[profile]["population"]
                depNum = round(depVal * routes["popSplit"] * pop)
                arrNum = round(arrVal * routes["popSplit"] * pop)
                depRate = depNum / arrivalTime[profile]["time"]
                arrRate = arrNum / arrivalTime[profile]["time"]
                precomputedRates[profile][routeId][bucket] = (depRate, arrRate)

    # Main simulation loop (per second)
    for curTime in range(startingTime, simulationTime):
        t = curTime - startingTime
        timeBucket = round_down(curTime, 3600)

        # Generate people
        for profile in departuresProfile:
            if profile not in routeTable:
                continue
            for routeId, routes in enumerate(routeTable[profile]):
                depRate, arrRate = precomputedRates[profile][routeId][timeBucket]

                if curTime < timeBucket + arrivalTime[profile]["time"]:
                    # Departures
                    numberOfDeparturesGenerated = depRate
                    profileNodePaths = routes["route"]
                    if profileNodePaths:
                        depAmount = math.floor(numberOfDeparturesGenerated)
                        if depAmount > 0:
                            peopleList.extend(
                                person(len(peopleList) + i, curTime, profileNodePaths, roadLengthTable, True)
                                for i in range(depAmount)
                            )

                    # Arrivals
                    numberOfArrivalsGenerated = arrRate
                    reversedProfileNode = routes["route"][::-1]
                    arrAmount = math.floor(numberOfArrivalsGenerated)
                    if arrAmount > 0:
                        peopleList.extend(
                            person(len(peopleList) + i, curTime, reversedProfileNode, roadLengthTable, True)
                            for i in range(arrAmount)
                        )

        # Update agents
        newPeopleList = []
        for agent in peopleList:
            if agent.agentInNewSpace:
                peopleCount[t, nodeIndex[agent.curNode]] += 1

            agent.updateAgentLocation(curTime)
            loc = agent.curNode
            if loc == "end":
                peopleEnd += 1
            else:
                occupancy[t, nodeIndex[loc]] += 1
                newPeopleList.append(agent)

        occupancy[t, nodeIndex["end"]] = peopleEnd
        peopleList = newPeopleList

    # Vectorized flow rate + width flow calc
    flowRate[flowRateBucket + 1 :] = peopleCount[1:-flowRateBucket] - peopleCount[:-flowRateBucket - 1]
    widthFlow[1:] = flowRate[:-1] / flowRateWidthTest

    # Save outputs with Pandas
    time = np.arange(startingTime, simulationTime)

    def save_csv(arr, name):
        df = pd.DataFrame(arr, columns=nodes)
        df.insert(0, "time", time)
        df.to_csv(f"UptownOutputs/{metro}/roadCalcs_{name}_{scenario}.csv", index=False)

    save_csv(occupancy, "OccupancyOutput")
    save_csv(peopleCount, "PeopleCountOutput")
    save_csv(flowRate, "FlowRateOutput")
    save_csv(widthFlow, "WidthFlowRequiredOutput")


In [ ]:
scenarioList = ['00','01','01M','02','02B','02BM','02M','03','04','04B','04BM','05','05B','A','AA','asianG','BBCC','BC','D','DM','esport','FIFA','megaSport','megaSportM','Olympics','OlympicsT','Opening','OpeningM']
metroList = ['pre','post']
for metro in metroList:
    for scenario in scenarioList:
        print(f"Running scenario {scenario} for metro {metro}")
        resetParams()
        readRouteTable(metro)
        readRoadLengthTable()
        readArrivalTime(scenario)
        readArrivalDepartureProfile(scenario)
        readArrivalArrivalProfile(scenario)
        runScenario(metro,scenario)

Running scenario 00 for metro pre
Running scenario 01 for metro pre
Running scenario 01M for metro pre
Running scenario 02 for metro pre
Running scenario 02B for metro pre
Running scenario 02BM for metro pre
Running scenario 02M for metro pre
